# Trabajo 3: Análisis de datos con NumPy y Pandas – RetailNow

Estructura del proyecto:

./
├── analisis_red_tiendas.ipynb
└── workspace/
    ├── sales.csv
    ├── inventories.csv
    └── satisfaction.csv


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)


## 1) Carga de datos (./workspace/)

In [ ]:
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "workspace"

SALES_PATH = DATA_DIR / "sales.csv"
INV_PATH   = DATA_DIR / "inventories.csv"
SAT_PATH   = DATA_DIR / "satisfaction.csv"

for p in [SALES_PATH, INV_PATH, SAT_PATH]:
    if not p.exists():
        raise FileNotFoundError(f"No se encontró {p}")

sales_raw = pd.read_csv(SALES_PATH)
inv_raw   = pd.read_csv(INV_PATH)
sat_raw   = pd.read_csv(SAT_PATH)

sales = sales_raw.dropna().copy()
inv   = inv_raw.dropna().copy()
sat   = sat_raw.dropna().copy()

print("Datos cargados correctamente")
print("Sales:", sales.shape)
print("Inventories:", inv.shape)
print("Satisfaction:", sat.shape)


## 2) Preparación de datos

In [ ]:
COL_TIENDA   = "ID_Tienda"
COL_PRODUCTO = "Producto"
COL_CANT     = "Cantidad_Vendida"
COL_PRECIO   = "Precio_Unitario"
COL_STOCK    = "Stock_Disponible"
COL_SAT      = "Satisfacción_Promedio"

sales[COL_CANT]   = pd.to_numeric(sales[COL_CANT], errors="coerce")
sales[COL_PRECIO] = pd.to_numeric(sales[COL_PRECIO], errors="coerce")
inv[COL_STOCK]    = pd.to_numeric(inv[COL_STOCK], errors="coerce")
sat[COL_SAT]      = pd.to_numeric(sat[COL_SAT], errors="coerce")

sales.dropna(inplace=True)
inv.dropna(inplace=True)
sat.dropna(inplace=True)

sales["Total_Ventas"] = sales[COL_CANT] * sales[COL_PRECIO]


## 3) Análisis de ventas

In [ ]:
ingresos_por_tienda = (
    sales.groupby(COL_TIENDA, as_index=False)["Total_Ventas"]
    .sum()
    .rename(columns={"Total_Ventas": "Ingresos_Totales"})
)

ingresos_por_tienda


## 4) Análisis de inventarios

In [ ]:
ventas_unidades = (
    sales.groupby([COL_TIENDA, COL_PRODUCTO], as_index=False)[COL_CANT]
    .sum()
    .rename(columns={COL_CANT: "Unidades_Vendidas"})
)

inv_analisis = inv.merge(
    ventas_unidades,
    on=[COL_TIENDA, COL_PRODUCTO],
    how="left"
)

inv_analisis["Unidades_Vendidas"].fillna(0, inplace=True)

inv_analisis["Rotacion_Inventario"] = np.where(
    inv_analisis[COL_STOCK] > 0,
    inv_analisis["Unidades_Vendidas"] / inv_analisis[COL_STOCK],
    np.nan
)

inv_analisis


## 5) Inventario crítico (<10%)

In [ ]:
inventario_critico = inv_analisis[
    inv_analisis["Rotacion_Inventario"] < 0.10
]

inventario_critico


## 6) Satisfacción del cliente

In [ ]:
satisfaccion_tienda = (
    sat.groupby(COL_TIENDA, as_index=False)[COL_SAT]
    .mean()
    .rename(columns={COL_SAT: "Satisfaccion_Media"})
)

satisfaccion_tienda


## 7) Operaciones con NumPy

In [ ]:
ventas_array = ingresos_por_tienda["Ingresos_Totales"].to_numpy()

mediana_ventas = np.median(ventas_array)
desviacion_ventas = np.std(ventas_array)

mediana_ventas, desviacion_ventas


## 8) Simulación de ventas futuras

In [ ]:
np.random.seed(42)

n_meses = 6
variacion = 0.10

proyecciones = ventas_array.reshape(-1, 1) * (
    1 + np.random.normal(0, variacion, size=(ventas_array.shape[0], n_meses))
)

proyecciones
